In [1]:
import joblib

# Örnek: Eğer kaydettiysen bu şekilde yüklenir
X_test = joblib.load("models/X_test.pkl")
y_test = joblib.load("models/y_val.pkl")
y_test = y_test.values if hasattr(y_test, "values") else y_test


In [2]:
import joblib
import numpy as np

ensemble_dict = joblib.load("models/ensemble_info.pkl")
weights = ensemble_dict["weights"]
mlp_preds = ensemble_dict["mlp_preds"]
y_scaler = joblib.load("models/y_scaler.pkl")

mlp_preds = y_scaler.inverse_transform(mlp_preds.reshape(-1, 1)).flatten()
# gerçek değer aralığına göre kırpılır
mlp_preds= np.clip(mlp_preds, y_test.min(), y_test.max())

lgbm_preds = ensemble_dict["lgbm_preds"]
rf_preds = ensemble_dict["rf_preds"]

#test gerçek değerleri
y_test = joblib.load("models/y_val.pkl")


In [3]:
print("MLP preds min/max:", mlp_preds.min(), mlp_preds.max())
print("LGBM preds min/max:", lgbm_preds.min(), lgbm_preds.max())
print("RF preds min/max:", rf_preds.min(), rf_preds.max())
print("y_test min/max:", y_test.min(), y_test.max())


MLP preds min/max: 10.66 99.95
LGBM preds min/max: 12.166994767665761 99.42620688714403
RF preds min/max: 13.910528402795466 98.10999848913471
y_test min/max: 10.66 99.95


In [4]:
w1, w2, w3 = weights
ensemble_preds = w1 * mlp_preds + w2 * lgbm_preds + w3 * rf_preds


In [5]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def smape(y_true, y_pred):
    return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_pred) + np.abs(y_true)))

mae = mean_absolute_error(y_test, ensemble_preds)
rmse = np.sqrt(mean_squared_error(y_test, ensemble_preds))
r2 = r2_score(y_test, ensemble_preds)
mape = np.mean(np.abs((y_test - ensemble_preds) / y_test)) * 100
smape_val = smape(y_test, ensemble_preds)
corr = np.corrcoef(y_test, ensemble_preds)[0, 1]

print(f"Ensemble Test Sonuçları:\n"
      f"MAE: {mae:.4f}\n"
      f"RMSE: {rmse:.4f}\n"
      f"R²: {r2:.4f}\n"
      f"MAPE: {mape:.2f}%\n"
      f"SMAPE: {smape_val:.2f}%\n"
      f"Pearson Corr: {corr:.4f}")


Ensemble Test Sonuçları:
MAE: 1.8847
RMSE: 2.6989
R²: 0.9610
MAPE: 3.78%
SMAPE: 3.69%
Pearson Corr: 0.9803


In [6]:
print("İlk 10 tahmin:", ensemble_preds[:10].round(2))
print("İlk 10 gerçek:", y_test[:10].round(2))


İlk 10 tahmin: [64.59 64.04 66.73 66.83 68.45 76.17 80.79 75.89 71.46 63.74]
İlk 10 gerçek: 25386    65.06
25387    66.30
25388    66.59
25389    66.94
25390    72.32
25391    79.92
25392    79.24
25393    77.07
25394    69.33
25395    62.12
Name: target_price, dtype: float64


### Test Sonuçları Üzerine Değerlendirme

Modeli test seti üzerinde çalıştırdım ve sonuçlar genel olarak oldukça iyi çıktı. Farklı algoritmalardan (MLP, LightGBM, Random Forest) oluşan ensemble model, bu test verisinde hem düşük hata verdi hem de yüksek korelasyon gösterdi.

Aldığım metrikler:

- **MAE:** 1.88
- **RMSE:** 2.70
- **R²:** 0.961
- **MAPE:** %3.78
- **SMAPE:** %3.69
- **Pearson Korelasyonu:** 0.980

Bu sonuçlara göre model gerçek değerlerle oldukça uyumlu tahminler yapmış. RMSE ve MAE değerleri düşük, ayrıca R² ve korelasyon gibi ölçütler 0.96–0.98 civarında. Bu da modelin hem hatasının az olduğunu hem de fiyat hareketlerini iyi takip ettiğini gösteriyor.

MAPE ve SMAPE değerlerinin de %4’ün altında olması, fiyat tahmini gibi zor bir problem için gayet başarılı sayılabilir. Tabii ki bazı uç noktalar veya ani fiyat değişimlerinde model hala zorlanabilir, ama genel olarak sonuçlar tatmin edici.

Kısacası: ensemble yaklaşımı işe yaramış görünüyor ve model sadece eğitime değil, gerçek test verisine de iyi genelleyebiliyor.
